In [7]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Read the `spacex_launch_geo.csv`
spacex_df = pd.read_csv('spacex_launch_geo.csv')

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Create a folium Map object with an initial center location at NASA Johnson Space Center in Houston, Texas
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

# Create a map object for launch sites
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define the locations and names of launch site markers
marker_locations = [
    {'location': [28.562302, -80.577356], 'name': 'CCAFS LC-40'},
    {'location': [28.563197, -80.576820], 'name': 'CCAFS SLC-40'},
    {'location': [28.573255, -80.646895], 'name': 'KSC LC-39A'},
    {'location': [34.632834, -120.610746], 'name': 'VAFB SLC-4E'}
]

# Add markers to the map with labels
for marker in marker_locations:
    folium.Marker(
        location=marker['location'],
        tooltip=marker['name'],
        popup=marker['name']
    ).add_to(site_map)

# Define the locations and properties of circle markers
circle_locations = [
    {'location': [28.562302, -80.577356], 'radius': 1000, 'color': 'blue', 'fill': True},
    {'location': [28.563197, -80.576820], 'radius': 1000, 'color': 'red', 'fill': True},
    {'location': [28.573255, -80.646895], 'radius': 1000, 'color': 'green', 'fill': True},
    {'location': [34.632834, -120.610746], 'radius': 1000, 'color': 'orange', 'fill': True},
]

# Add circle markers to the map
for circle in circle_locations:
    folium.Circle(
        location=circle['location'],
        radius=circle['radius'],
        color=circle['color'],
        fill=circle['fill']
    ).add_to(site_map)

# Create a marker cluster for launch outcomes
marker_cluster = MarkerCluster()

# Function to assign color to launch outcome based on 'class' column
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Add 'marker_color' column to spacex_df based on launch outcome
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Add markers to the marker cluster with colored icons based on launch outcome
for lat, lng, label, mark in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['Launch Site'], spacex_df['marker_color']):
    fol = folium.Marker(location=[lat, lng], icon=folium.Icon(color='orange', icon_color=mark,)).add_to(marker_cluster)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Calculate distance between two points on Earth's surface using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate distance from a launch site to the closest coastline
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_lat = 28.56293
coastline_lon = -80.56785
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a marker for the closest coastline point and display the distance on the icon
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline),
    )
)

# Add coastline marker and a polyline to the map
site_map.add_child(distance_marker)
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)], weight=1).add_to(site_map)

# Create a marker for a point of interest (e.g., railway) and display the distance on the icon
poi_lat = 28.57223
poi_lon = -80.58525
distance_poi = calculate_distance(launch_site_lat, launch_site_lon, poi_lat, poi_lon)
poi_marker = folium.Marker(
    [poi_lat, poi_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_poi),
    )
)

# Add the point of interest marker and a polyline to the map
site_map.add_child(poi_marker)
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (poi_lat, poi_lon)], color='pink').add_to(site_map)

# Display the map
site_map
